<a href="https://colab.research.google.com/github/siddugoud6966/NLP_2024-2025/blob/main/NLP_ASS_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]
(i) Build the Transformer Model on above dataset


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

input_ids = tokens['input_ids']

print(f"Tokenized input ids: {input_ids}")

model = GPT2LMHeadModel.from_pretrained('gpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output = model(input_ids.to(device), labels=input_ids.to(device))
loss = output.loss
logits = output.logits

print(f"Loss: {loss.item()}")

Tokenized input ids: tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,  1310,  2576,
          3706,  2297, 36032, 17233,    13,  1375,  6151,   284,  3187,   607,
         18410,    11,   508,  5615,   287,   262, 16479,    13,  1881,  1110,
            11,   607,  2802,  1965,   607,   284,  1011,   257,  7988,   286,
         39863,   284,   607, 18410,    13,  1550,   607,   835,   832,   262,
         16479,    11,   673,  1138,   257,  1263,  2089, 17481,   508,  2227,
           284,  4483,   607,    13]])
Loss: 2.5232996940612793


(ii) Train the model using 20, 60, 70 epochs

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add the padding token to the Tokenizer
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))

# Example text data
text_data = [
    "Once upon a time, there was a little girl named Red Riding Hood.",
    "She loved to visit her grandmother, who lived in the woods.",
    "One day, her mother asked her to take a basket of goodies to her grandmother."
    # Add more sentences or text for a richer dataset
]

# Create a Dataset from the text data
data = {"text": text_data}
dataset = Dataset.from_dict(data)

# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=50)

# Tokenize and format the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Training function
def train_model(num_epochs):
    training_args = TrainingArguments(
        output_dir=f"./results_{num_epochs}_epochs",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=1,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=10,
        save_total_limit=2,
        report_to="none"
    )

    class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            # Generate labels from input_ids shifting by 1
            labels = inputs.get("input_ids").clone()
            labels[labels == tokenizer.pad_token_id] = -100 # Ignore padded tokens in loss computation
            # Forward pass with labels to compute loss
            outputs = model(**inputs, labels=labels)
            # Return the loss
            return outputs.loss

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )

    trainer.train()
    print(f"Training completed for {num_epochs} epochs.")

# Train the model with 20, 60, and 70 epochs
train_model(20)
train_model(60)
train_model(70)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Step,Training Loss
10,2.512800
20,0.670400
30,0.286700
40,0.153600
50,0.213500
60,0.188000


Training completed for 20 epochs.


Step,Training Loss
10,0.144700
20,0.067800
30,0.144500
40,0.114600
50,0.159700
60,0.146300
70,0.128000
80,0.043900
90,0.089700
100,1.381100


Training completed for 60 epochs.


Step,Training Loss
10,0.063000
20,0.004700
30,0.024700
40,0.015200
50,0.053600
60,0.003100
70,0.000500
80,0.005400
90,0.000800
100,0.002700


Training completed for 70 epochs.


 (iii) After training, use the model to generate new text by feeding it an initial seed text



In [ ]:
import torch
import warnings
from transformers import GPT2Tokenizer, GPT2LMHeadModel

warnings.filterwarnings("ignore", message="Setting `pad_token_id` to `eos_token_id`")

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

seed_text = "Once upon a time"

input_ids = tokenizer.encode(seed_text, return_tensors="pt")
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setti

Once upon a time, the world was a place of great beauty and great danger. The world of the gods was the place where the great gods were born, and where they were to live.

The world that was created was not the same


(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.



In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

input_ids = tokens['input_ids']
attention_mask = tokens.get('attention_mask', torch.ones_like(input_ids))

print(f"Tokenized input ids: {input_ids}")

model = GPT2LMHeadModel.from_pretrained('gpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=input_ids.to(device))
loss = output.loss
logits = output.logits

print(f"Loss: {loss.item()}")

model.eval()
generated_text = model.generate(
    input_ids.to(device),
    attention_mask=attention_mask.to(device),
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    temperature=0.7,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(f"Generated Text: {generated_text}")

Tokenized input ids: tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,  1310,  2576,
          3706,  2297, 36032, 17233,    13,  1375,  6151,   284,  3187,   607,
         18410,    11,   508,  5615,   287,   262, 16479,    13,  1881,  1110,
            11,   607,  2802,  1965,   607,   284,  1011,   257,  7988,   286,
         39863,   284,   607, 18410,    13,  1550,   607,   835,   832,   262,
         16479,    11,   673,  1138,   257,  1263,  2089, 17481,   508,  2227,
           284,  4483,   607,    13]])
Loss: 2.5232996940612793
Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. She decided to take her mother's basket of goodies. When Red Riding Hood came down the trail to the other side of the trail, she was surprised to 